# 디퓨전 모델로부터 샘플 추출하기

이 노트북에서 우리는 이전에 학습된 디퓨전 모델로부터 샘플을 추출할 것입니다.
- DDPM과 DDIM sampler로부터 추출한 샘플을 비교합니다
- 추출된 샘플을 조건에 따라 시각화합니다

DDPM과 DDIM은 이미지 생성 모델들입니다.  
전자는 처리 시간은 오래 걸리지만 성능이 좋고, 후자는 처리 시간이 짧지만 성능이 안좋습니다.

In [ ]:
from pathlib import Path
from types import SimpleNamespace
import torch
import torch.nn.functional as F
import numpy as np
from utilities import *

import wandb

In [ ]:
wandb.login(anonymous="allow")

# 세팅

In [ ]:
# Wandb 파라미터
MODEL_ARTIFACT = "dlai-course/model-registry/SpriteGen:latest"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

config = SimpleNamespace(
    # 하이퍼파라미터
    num_samples = 30,
    
    # ddpm sampler 하이퍼파라미터
    timesteps = 500,
    beta1 = 1e-4,
    beta2 = 0.02,
    
    # ddim sampler 하이퍼파라미터
    ddim_n = 25,
    
    # 네트워크 하이퍼파라미터
    height = 16,
)

이전 노트북에서 우리는 가장 성능이 좋았던 모델을 wandb Artifact로 저장했습니다.(학습중에 파일을 저장하는 방식)  
이제 그 모델을 wandb로부터 불러오고 sampling loop를 세팅해봅시다

In [ ]:
def load_model(model_artifact_name):
    "wandb artifacts에서 모델을 불러옵니다"
    api = wandb.Api()
    artifact = api.artifact(model_artifact_name, type="model")
    model_path = Path(artifact.download())

    # 레지스트리로부터 모델 정보를 복원합니다
    producer_run = artifact.logged_by()

    # weight 딕셔너리를 불러옵니다
    model_weights = torch.load(model_path/"context_model.pth", 
                               map_location="cpu")

    # 모델을 생성합니다
    model = ContextUnet(in_channels=3, 
                        n_feat=producer_run.config["n_feat"], 
                        n_cfeat=producer_run.config["n_cfeat"], 
                        height=producer_run.config["height"])
    
    # 불러온 weight를 모델에 전달합니다
    model.load_state_dict(model_weights)

    # 모델을 평가 모드로 전환합니다
    model.eval()
    return model.to(DEVICE)

In [ ]:
nn_model = load_model(MODEL_ARTIFACT)

## 샘플 추출

우리는 샘플을 추출하고 생성된 샘플을 wandb에 로그로 남길 것입니다

In [ ]:
_, sample_ddpm_context = setup_ddpm(config.beta1, 
                                    config.beta2, 
                                    config.timesteps, 
                                    DEVICE)

노이즈와 context vector를 정의합니다

In [ ]:
# Noise vector
# x_T ~ N(0, 1), 초기 노이즈를 샘플링합니다
noises = torch.randn(config.num_samples, 3, 
                     config.height, config.height).to(DEVICE)  

# 고정된 context vector는 다음으로부터 추출합니다
ctx_vector = F.one_hot(torch.tensor([0,0,0,0,0,0,   # hero
                                     1,1,1,1,1,1,   # non-hero
                                     2,2,2,2,2,2,   # food
                                     3,3,3,3,3,3,   # spell
                                     4,4,4,4,4,4]), # side-facing 
                       5).to(DEVICE).float()

속도가 빠른 DDIM sampler를 불러와 봅시다

In [ ]:
sample_ddim_context = setup_ddim(config.beta1, 
                                 config.beta2, 
                                 config.timesteps, 
                                 DEVICE)

### 샘플 추출:
이전과 마찬가지로 ddpm 샘플을 작동시킵니다

In [ ]:
ddpm_samples, _ = sample_ddpm_context(nn_model, noises, ctx_vector)

DDIM에 대해서는 `n` 파라미터로 스텝 사이즈를 조절할 수 있습니다:

In [ ]:
ddim_samples, _ = sample_ddim_context(nn_model, 
                                      noises, 
                                      ctx_vector, 
                                      n=config.ddim_n)

### 생성 결과를 테이블로 시각화하기
생성 결과를 저장할 수 있는 `wandb.Table` 를 만들어 봅시다

In [ ]:
table = wandb.Table(columns=["input_noise", "ddpm", "ddim", "class"])

표의 줄을 하나씩 추가할수도 있고, `wandb.Image`를 이용하여 이미지를 전달하면 렌더링됩니다

In [ ]:
for noise, ddpm_s, ddim_s, c in zip(noises, 
                                    ddpm_samples, 
                                    ddim_samples, 
                                    to_classes(ctx_vector)):
    
    # 표에 데이터를 각 행별로 추가합니다
    table.add_data(wandb.Image(noise),
                   wandb.Image(ddpm_s), 
                   wandb.Image(ddim_s),
                   c)

context 매니저로 `wandb.init`을 사용하고, 표를 W&B에 로깅할 수 있습니다.  
이전과 마찬가지로 run이 끝나면 종료해줘야 합니다(with 구문을 사용하면 자동으로 종료해줍니다 - 파일을 닫아줍니다)

In [ ]:
with wandb.init(project="dlai_sprite_diffusion", 
                job_type="samplers_battle", 
                config=config):
    
    wandb.log({"samplers_table":table})